# Build json file from folders

## Create thumbnails in jpg & fix metadata. Series only contain one item

In [1]:
from PIL import Image
import json
import glob
import re
import pandas as pd
import os

In [6]:
csv_file='building.csv'
output_file="../../building.json"
image_types=('jpg', 'jpeg', 'png', 'tif')
csv_separator=';'

In [7]:
current_directory=os.getcwd() 

df=pd.read_csv(current_directory + '/' + csv_file, sep=csv_separator)  
door_width=1
door_height=2

building={}
for i,row in df.iterrows():
    building[df["Gallery name"][i]]={}
    building[df["Gallery name"][i]]["geometry"]=df.loc[i, 'width': '#items E'].values.astype(int).tolist()
    building[df["Gallery name"][i]]["parent"]=df.loc[i, 'Parent']
    son_galleries=sum(df.Parent==df["Gallery name"][i])
    #if has son galleries
    if (son_galleries >0):
        for j, jrow in df.loc[df["Parent"]=="root"].iterrows():
            building[df["Gallery name"][i]][jrow["Gallery name"]]={}
            building[df["Gallery name"][i]][jrow["Gallery name"]]["resource"]=jrow["Gallery name"] +'.glb'
            building[df["Gallery name"][i]][jrow["Gallery name"]]["resource_type"]='door'
            building[df["Gallery name"][i]][jrow["Gallery name"]]["width"]= door_width
            building[df["Gallery name"][i]][jrow["Gallery name"]]["height"]= door_height
    else:
        files_grabbed = []
        for image_type in image_types:
            files_grabbed.extend(glob.glob(current_directory + df["Folder"][i] + '/*.' + image_type))
        for file_name in files_grabbed:
            im=Image.open(file_name)
            width, height = im.size
            as_ratio=height/width
            if (as_ratio<=1):
                width=1
                height=as_ratio
            else:
                width=1/as_ratio
                height=1

            possible_extensions= ''.join(image_type + '|' for image_type in image_types)
            possible_extensions = possible_extensions[:-1]
            name=re.search(r'.*\\(.*).(' + possible_extensions + ')',file_name)[1]
            file_name_server=file_name.replace('\\','/')
            file_name_server=df["Folder"][i] + '/' + re.search(r'.*/(.*)', file_name_server)[1]

            building[df["Gallery name"][i]][name]= {"resource": file_name_server, "resource_type": 'image',"width": f'{width:.2f}', "height": f'{height:.2f}'}


    
with open(current_directory + '/' + output_file, "w") as fp:
    json.dump(building,fp, indent=2)